In [1]:
# imports
import pandas as pd
import numpy as np
import seaborn as sns


In [2]:
# # sample code to read in pivot table 
# # yay stackoverflow!
# # https://stackoverflow.com/questions/55359807/save-pandas-pivot-table-to-include-index-and-columns-names

# utility_matrix = pd.read_csv('../data/user_rating_matrix.csv', index_col=0)
# utility_matrix.columns.name = utility_matrix.index.name
# utility_matrix.index.name = 'title'
# utility_matrix.head()

In [3]:
#datasets
movies = pd.read_csv("../../data/ml-latest-small/movies.csv")
ratings = pd.read_csv("../../data/ml-latest-small/ratings.csv")

In [4]:
#clean and proper mapping of movies
movies.sort_values(by='movieId', ascending=True, inplace=True)
ratings.sort_values(by='movieId', ascending=True, inplace=True)
movies.reset_index(inplace=True, drop=True)

movie_di = dict()

for index, row in movies.iterrows():
    if row["movieId"] not in movie_di:
        movie_di[row["movieId"]] = index
        
# update rows
movies = movies.replace({"movieId": movie_di})
ratings = ratings.replace({"movieId": movie_di})

In [5]:
movies.head(-10)


,movieId,title,genres
0,0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,Jumanji (1995),Adventure|Children|Fantasy
2,2,Grumpier Old Men (1995),Comedy|Romance
3,3,Waiting to Exhale (1995),Comedy|Drama|Romance
4,4,Father of the Bride Part II (1995),Comedy
...,...,...,...
9727,9727,John From (2015),Drama
9728,9728,Liquid Truth (2017),Drama
9729,9729,Bunny (1998),Animation
9730,9730,Hommage à Zgougou (et salut à Sabine Mamou) (2...,Documentary


In [6]:
ratings.head(-10)

,userId,movieId,rating,timestamp
0,1,0,4.0,964982703
81531,517,0,4.0,1487954343
30517,213,0,3.5,1316196157
81082,514,0,4.0,1533872400
30601,214,0,3.0,853937855
...,...,...,...,...
52005,338,9727,1.0,1530148478
52006,338,9728,1.5,1530148477
52007,338,9729,1.0,1530148473
52008,338,9730,1.0,1530148473


In [9]:
movies_ratings = pd.merge(movies, ratings, on = 'movieId')

utility_matrix = movies_ratings.pivot_table(values = 'rating', index = 'title', columns = 'userId').fillna(0) 

utility_matrix.to_csv('../../data/user_rating_matrix.csv', index_label=utility_matrix.columns.name)

utility_matrix.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Testing based on tags 
*what other features could you test?*

In [ ]:
# adding colums, binary 0,1 for if the tag exists
all_tags = {
    "Action",
    "Adventure",
    "Animation",
    "Children's",
    "Comedy",
    "Crime",
    "Documentary",
    "Drama",
    "Fantasy",
    "Film-Noir",
    "Horror",
    "Musical",
    "Mystery",
    "Romance",
    "Sci-Fi",
    "Thriller",
    "War",
    "Western",
#     "(no genres listed)" # not needed - just all 0's
}

for t in all_tags :
    ratings[f'Tag_{t}'] = False # default initialize 0


In [8]:
# string s, return list of tags
def get_tags(s) : 
    return s.strip().split("|")

In [9]:
# assignment
for index, mov_id in enumerate(ratings["movieId"]) :
    for tag in get_tags(movies["genres"][mov_id]) :
        if tag in all_tags :
            ratings.loc[index, f'Tag_{tag}'] = True
            
# Note: this took me about a minute to run

In [10]:
# Note that there are 232 indices fo userId 1, so that's why it looks out of order...
ratings.sort_values(by='userId', ascending=True, inplace=True)
# ratings.head() 

In [11]:
ratings.dtypes

userId               int64
movieId              int64
rating             float64
timestamp            int64
Tag_Sci-Fi            bool
Tag_Drama             bool
Tag_Mystery           bool
Tag_Romance           bool
Tag_Film-Noir         bool
Tag_War               bool
Tag_Thriller          bool
Tag_Western           bool
Tag_Documentary       bool
Tag_Animation         bool
Tag_Adventure         bool
Tag_Comedy            bool
Tag_Action            bool
Tag_Crime             bool
Tag_Musical           bool
Tag_Children's        bool
Tag_Horror            bool
Tag_Fantasy           bool
dtype: object

In [12]:
ratings.head()

,userId,movieId,rating,timestamp,Tag_Sci-Fi,Tag_Drama,Tag_Mystery,Tag_Romance,Tag_Film-Noir,Tag_War,...,Tag_Documentary,Tag_Animation,Tag_Adventure,Tag_Comedy,Tag_Action,Tag_Crime,Tag_Musical,Tag_Children's,Tag_Horror,Tag_Fantasy
0,1,0,4.0,964982703,False,False,False,False,False,False,...,False,True,True,True,False,False,False,False,False,True
115,1,1431,5.0,964982176,False,False,False,False,False,False,...,False,False,True,False,False,False,True,False,False,True
1,1,2,4.0,964981247,False,True,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
2,1,5,4.0,964982224,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
3,1,43,5.0,964983815,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False


## test/train

In [13]:
from sklearn.model_selection import train_test_split

X = ratings[[
            'movieId',
#              'userId',
             "Tag_Action", 
             "Tag_Adventure", 
             "Tag_Animation", 
             "Tag_Children's",
             "Tag_Comedy",
             "Tag_Crime",
             "Tag_Documentary",
             "Tag_Drama",
             "Tag_Fantasy",
             "Tag_Film-Noir",
             "Tag_Horror",
             "Tag_Musical",
             "Tag_Mystery",
             "Tag_Romance",
             "Tag_Sci-Fi",
             "Tag_Thriller",
             "Tag_War",
             "Tag_Western"
           ]] # choose features for training data set
y = ratings['rating']  # output

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)


## Do your own

## See how scikit fares

In [14]:
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error as MSE

In [15]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=17)
knn.fit(X_train,y_train)

y_sklearn_pred = knn.predict(X_test)


In [ ]:
y_test_update = [int(n*2) for n in y_test]
y_sklearn_pred_update = [int(n*2) for n in y_sklearn_pred]

print(classification_report(y_test_update, y_sklearn_pred_update))


# note that instead of .5 - 5, this is mapped from 1 - 10

In [17]:
MSE(y_test, y_sklearn_pred)

0.9862222212309335

In [18]:
for x,y in (zip(y_sklearn_pred, y_test)) :
    print(x,y)

3.4705882352941178 4.0
3.764705882352941 5.0
4.147058823529412 5.0
3.5294117647058822 4.0
3.3529411764705883 4.0
3.764705882352941 3.0
3.0294117647058822 3.5
4.352941176470588 2.0
3.235294117647059 3.0
3.5588235294117645 2.0
3.9705882352941178 3.5
4.117647058823529 3.0
2.911764705882353 2.5
4.294117647058823 5.0
3.7941176470588234 2.0
4.294117647058823 3.5
3.411764705882353 5.0
3.2941176470588234 1.0
3.5 3.0
3.5588235294117645 4.0
3.088235294117647 4.5
4.088235294117647 5.0
2.735294117647059 4.5
4.264705882352941 4.5
3.9705882352941178 3.5
3.6470588235294117 3.0
3.676470588235294 3.0
3.3529411764705883 3.0
2.5 4.0
3.2058823529411766 4.5
3.5588235294117645 4.5
4.235294117647059 4.0
3.5588235294117645 4.0
4.264705882352941 5.0
3.8529411764705883 4.0
3.7941176470588234 3.5
3.8823529411764706 4.0
3.1470588235294117 1.5
4.147058823529412 3.5
3.7941176470588234 3.5
3.2058823529411766 2.0
3.735294117647059 3.0
3.5 3.0
3.264705882352941 5.0
4.0588235294117645 3.5
3.9705882352941178 4.0
3.94117

3.0294117647058822 3.0
3.2058823529411766 3.0
3.735294117647059 3.5
3.5294117647058822 3.5
2.2941176470588234 0.5
3.8529411764705883 3.5
2.735294117647059 2.0
3.5588235294117645 4.5
3.823529411764706 4.0
3.6176470588235294 4.0
4.382352941176471 4.0
2.764705882352941 3.5
3.411764705882353 4.0
3.764705882352941 4.0
4.411764705882353 4.0
3.7058823529411766 4.0
3.3823529411764706 0.5
3.764705882352941 5.0
3.0588235294117645 4.5
4.029411764705882 3.0
3.0294117647058822 3.5
3.9705882352941178 4.0
2.411764705882353 3.0
3.9411764705882355 4.0
3.1176470588235294 3.5
3.911764705882353 3.5
3.823529411764706 5.0
2.9705882352941178 0.5
3.1176470588235294 4.0
4.029411764705882 4.0
3.3823529411764706 2.5
2.9411764705882355 3.5
3.4411764705882355 5.0
3.8823529411764706 4.0
4.147058823529412 2.0
3.6176470588235294 3.0
3.3529411764705883 2.0
2.5 2.5
3.411764705882353 4.5
3.3823529411764706 3.0
3.676470588235294 2.0
3.4705882352941178 4.5
3.2941176470588234 4.0
2.588235294117647 2.0
3.176470588235294 2.0

3.323529411764706 1.0
3.676470588235294 2.5
2.5294117647058822 3.0
3.7941176470588234 4.0
3.2941176470588234 3.0
3.0588235294117645 3.0
3.1470588235294117 4.5
3.7941176470588234 4.0
3.911764705882353 4.5
3.735294117647059 3.5
3.8823529411764706 4.0
3.8529411764705883 5.0
4.411764705882353 4.5
4.0588235294117645 4.0
3.6470588235294117 4.0
3.1470588235294117 3.5
3.5 5.0
3.3823529411764706 4.0
3.3823529411764706 4.0
2.764705882352941 3.0
3.823529411764706 4.0
3.5294117647058822 2.0
3.7058823529411766 3.0
3.235294117647059 4.0
3.911764705882353 4.0
3.6470588235294117 4.5
4.0 5.0
4.117647058823529 4.0
3.5294117647058822 4.0
3.9705882352941178 4.5
3.5294117647058822 4.0
2.9411764705882355 1.0
3.5294117647058822 3.0
3.7058823529411766 4.0
2.911764705882353 2.5
4.029411764705882 5.0
3.764705882352941 4.5
4.0 4.0
3.5 4.5
3.7058823529411766 4.0
2.7941176470588234 2.0
3.2941176470588234 3.5
3.1470588235294117 3.0
3.2058823529411766 3.0
3.411764705882353 4.0
4.147058823529412 3.5
3.647058823529411

3.1176470588235294 3.0
3.235294117647059 5.0
2.8529411764705883 2.0
3.5 2.0
4.029411764705882 5.0
3.6470588235294117 4.5
3.6470588235294117 4.0
3.9411764705882355 4.0
3.9705882352941178 3.0
3.911764705882353 4.0
3.8823529411764706 3.0
2.735294117647059 0.5
3.4411764705882355 5.0
3.411764705882353 3.5
3.735294117647059 4.5
4.029411764705882 4.0
3.6176470588235294 5.0
3.588235294117647 4.5
4.4411764705882355 3.5
3.0294117647058822 4.5
3.3823529411764706 5.0
4.205882352941177 5.0
3.5 5.0
3.5588235294117645 3.5
2.9411764705882355 3.0
4.117647058823529 3.0
3.7941176470588234 3.5
3.823529411764706 1.0
3.735294117647059 3.0
2.823529411764706 3.0
3.735294117647059 5.0
3.7941176470588234 4.0
3.764705882352941 3.5
3.735294117647059 3.5
2.9411764705882355 4.5
2.7941176470588234 3.0
2.676470588235294 2.0
3.676470588235294 3.0
3.6470588235294117 4.0
3.2941176470588234 3.0
3.2941176470588234 3.0
2.588235294117647 4.0
3.5294117647058822 3.0
2.8823529411764706 3.0
3.911764705882353 4.0
3.9411764705882

3.9705882352941178 4.0
3.7058823529411766 4.0
3.1470588235294117 2.5
3.2941176470588234 4.0
4.176470588235294 1.0
3.5 3.0
4.029411764705882 4.0
3.4411764705882355 2.0
4.323529411764706 4.0
2.8823529411764706 1.0
3.0 2.5
2.823529411764706 4.0
3.5294117647058822 2.0
3.8823529411764706 4.5
4.176470588235294 4.0
3.911764705882353 4.0
3.7058823529411766 3.0
2.5294117647058822 3.0
3.3529411764705883 5.0
3.323529411764706 3.0
3.088235294117647 3.0
3.411764705882353 1.0
3.088235294117647 4.5
3.323529411764706 2.0
3.5294117647058822 4.0
3.3529411764705883 4.0
3.4705882352941178 3.5
3.7941176470588234 3.5
3.0294117647058822 1.0
3.823529411764706 5.0
3.5294117647058822 5.0
3.323529411764706 3.0
4.176470588235294 5.0
2.5588235294117645 1.0
3.323529411764706 3.5
4.0 5.0
3.6470588235294117 3.5
3.264705882352941 4.5
4.352941176470588 5.0
3.9411764705882355 5.0
3.2941176470588234 3.0
3.3529411764705883 3.5
3.823529411764706 4.0
2.6470588235294117 3.0
4.0 2.0
4.147058823529412 3.5
3.0 1.0
3.58823529411

...

2.735294117647059 4.0
4.323529411764706 5.0
3.5588235294117645 4.0
2.264705882352941 3.0
3.4411764705882355 5.0
3.8529411764705883 4.0
3.764705882352941 3.0
3.1176470588235294 2.0
3.3529411764705883 4.0
3.3529411764705883 4.0
3.5294117647058822 3.0
3.5294117647058822 4.0
3.235294117647059 3.0
4.0 4.5
3.823529411764706 3.0
4.411764705882353 4.0
3.8529411764705883 4.0
3.8823529411764706 3.5
3.588235294117647 4.0
4.088235294117647 4.0
3.7941176470588234 4.0
3.4705882352941178 4.0
2.9411764705882355 3.0
3.2941176470588234 4.0
3.8529411764705883 3.5
3.6176470588235294 4.0
2.5 4.0
4.264705882352941 4.0
3.4705882352941178 3.5
4.5 4.5
3.764705882352941 0.5
4.117647058823529 4.5
3.264705882352941 3.0
4.0 5.0
4.176470588235294 4.5
4.205882352941177 4.5
4.0 4.0
3.5294117647058822 4.0
4.352941176470588 5.0
3.5588235294117645 4.0
4.0 4.0
4.0 3.5
3.6470588235294117 5.0
3.8529411764705883 3.5
3.5 5.0
3.823529411764706 5.0
3.588235294117647 4.0
3.235294117647059 5.0
3.176470588235294 5.0
3.91176470588

3.088235294117647 4.0
3.735294117647059 4.0
3.8529411764705883 3.0
3.6176470588235294 4.5
3.323529411764706 1.5
4.0 3.5
3.735294117647059 3.0
3.8823529411764706 5.0
3.176470588235294 3.0
3.088235294117647 3.0
3.5294117647058822 3.0
2.6470588235294117 2.0
3.7941176470588234 1.0
3.735294117647059 5.0
3.4705882352941178 3.0
3.676470588235294 4.5
3.588235294117647 3.5
3.4411764705882355 4.0
3.9705882352941178 4.0
3.764705882352941 3.5
3.5588235294117645 2.0
3.676470588235294 4.0
4.117647058823529 5.0
3.4411764705882355 4.0
3.5588235294117645 2.0
3.9411764705882355 2.5
3.8823529411764706 4.0
3.7058823529411766 4.5
3.8823529411764706 2.5
3.6470588235294117 3.5
3.2058823529411766 5.0
3.2941176470588234 4.0
4.0 5.0
3.5 3.0
4.147058823529412 4.0
3.3823529411764706 3.0
3.6176470588235294 2.5
3.0588235294117645 3.0
3.3529411764705883 3.5
3.088235294117647 3.0
3.0588235294117645 2.0
4.0588235294117645 5.0
4.205882352941177 5.0
3.3529411764705883 3.5
3.6470588235294117 3.5
3.1470588235294117 3.5
3.

3.764705882352941 4.0
3.764705882352941 2.5
3.588235294117647 3.0
4.4411764705882355 5.0
3.5 3.5
4.117647058823529 2.0
3.8529411764705883 4.0
3.411764705882353 3.0
3.1470588235294117 3.5
3.1176470588235294 3.0
3.7058823529411766 5.0
3.3823529411764706 4.0
4.588235294117647 3.5
2.9411764705882355 5.0
3.911764705882353 4.0
3.4411764705882355 4.5
2.9411764705882355 3.5
3.0294117647058822 3.0
3.8823529411764706 4.0
3.4411764705882355 3.0
2.823529411764706 3.0
2.735294117647059 3.5
3.7058823529411766 0.5
3.411764705882353 3.0
4.088235294117647 4.0
3.676470588235294 3.5
3.264705882352941 4.0
4.294117647058823 4.0
2.911764705882353 3.5
3.4705882352941178 2.5
2.823529411764706 4.0
4.0588235294117645 3.0
4.147058823529412 5.0
3.735294117647059 3.0
3.2941176470588234 3.0
3.823529411764706 4.0
3.9411764705882355 3.0
3.676470588235294 4.0
2.6470588235294117 2.0
3.9411764705882355 3.0
4.352941176470588 1.0
2.8529411764705883 4.0
2.9705882352941178 3.0
3.823529411764706 3.0
3.5 2.5
3.705882352941176

4.0 4.0
3.676470588235294 3.0
3.0294117647058822 4.0
3.5 2.5
4.0588235294117645 5.0
3.9411764705882355 4.5
4.176470588235294 3.0
3.764705882352941 4.5
2.2941176470588234 2.0
3.8529411764705883 5.0
2.8823529411764706 3.0
3.735294117647059 4.0
3.5 2.5
2.588235294117647 3.5
3.323529411764706 4.0
3.1176470588235294 3.0
3.7058823529411766 4.0
4.205882352941177 5.0
2.6470588235294117 1.0
3.764705882352941 3.5
3.9705882352941178 3.0
3.1176470588235294 1.0
4.088235294117647 5.0
3.3529411764705883 2.5
3.3823529411764706 3.0
2.6470588235294117 3.0
2.7941176470588234 2.0
4.0 3.0
4.0588235294117645 4.5
4.0 4.0
3.0588235294117645 1.5
2.735294117647059 1.0
2.6176470588235294 3.5
3.1470588235294117 3.0
3.6176470588235294 3.0
3.764705882352941 4.5
3.4411764705882355 3.5
3.911764705882353 4.0
3.264705882352941 3.0
3.911764705882353 3.0
4.264705882352941 4.0
3.676470588235294 4.5
3.3529411764705883 4.0
3.2058823529411766 3.5
4.411764705882353 5.0
3.7058823529411766 3.5
3.2058823529411766 2.5
3.117647058

3.735294117647059 3.0
2.4411764705882355 3.5
3.7941176470588234 4.0
4.0588235294117645 5.0
3.588235294117647 4.5
3.323529411764706 4.0
3.1470588235294117 2.5
3.1470588235294117 2.0
3.5 3.0
2.911764705882353 4.0
2.911764705882353 4.0
3.5294117647058822 4.0
4.294117647058823 4.0
2.735294117647059 2.5
2.6176470588235294 2.0
3.735294117647059 5.0
4.235294117647059 4.5
3.4705882352941178 2.0
2.911764705882353 3.5
3.588235294117647 4.0
4.382352941176471 5.0
3.5588235294117645 3.0
3.7058823529411766 2.5
3.5294117647058822 3.0
3.323529411764706 4.0
3.764705882352941 4.0
3.1470588235294117 2.5
2.088235294117647 2.0
3.264705882352941 3.0
3.0588235294117645 5.0
3.6176470588235294 5.0
3.235294117647059 3.0
3.264705882352941 4.0
3.0294117647058822 4.0
3.9411764705882355 4.5
2.8529411764705883 4.0
3.6470588235294117 4.5
3.1176470588235294 4.0
3.9411764705882355 4.0
3.0 5.0
3.6176470588235294 5.0
3.411764705882353 4.0
4.088235294117647 5.0
3.264705882352941 4.0
3.5294117647058822 3.0
3.52941176470588